In [39]:
import numpy as np
import pandas as pd
from math import sqrt
import requests
from scipy.optimize import minimize
from itertools import combinations
import plotly.graph_objects as go

**1. Configuration/Parameters**

In [ ]:
API_KEY = "Upload_Your_Own_Key"

# Universe of stock tickers
universe_tickers = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "META", 
    "JNJ", "JPM", "V", "PG", "XOM", 
    "INTC", "KO", "WMT", "NVDA"
]

# Using SPY (ETF) as proxy for S&P 500
benchmark_ticker = "SPY"   

# Rolling 1-year lookback (252 trading days), monthly rebalancing (every 20 days)
lookback_period = 252
rebalance_interval = 5

# Portfolio size
portfolio_size = 8

# Max weight constraint per ticker
max_weight = 0.30

# Convert annual risk-free rate to daily
annual_rf= 0.04
risk_free_daily = (1 + annual_rf)**(1/252) - 1  # ~0.000156 (0.0156% daily)

**2. Fetch Data**

In [41]:
base_url = "https://www.alphavantage.co/query"
function_daily_adjusted = "TIME_SERIES_DAILY_ADJUSTED"

def fetch_alpha_vantage_data(symbol, api_key=API_KEY):
    """
    Fetch daily adjusted price data for the given symbol from Alpha Vantage
    and return a Series with 'Adj Close' prices (indexed by date).
    """
    params = {
        "function": function_daily_adjusted,
        "symbol": symbol,
        "apikey": api_key,
        "outputsize": "full"
    }
    resp = requests.get(base_url, params=params)
    # If using the free tier for many tickers, consider adding: time.sleep(15) between requests

    if resp.status_code != 200:
        print(f"[Error {resp.status_code}] Could not fetch data for {symbol}.")
        return pd.Series(dtype=float)

    data_json = resp.json()
    if "Time Series (Daily)" not in data_json:
        print(f"No 'Time Series (Daily)' in API response for {symbol}.")
        return pd.Series(dtype=float)

    ts = data_json["Time Series (Daily)"]
    df_symbol = pd.DataFrame.from_dict(ts, orient="index")
    df_symbol.index = pd.to_datetime(df_symbol.index)
    df_symbol.sort_index(inplace=True)
    # '5. adjusted close' is the Adjusted Close
    df_symbol["Adj Close"] = df_symbol["5. adjusted close"].astype(float)
    return df_symbol["Adj Close"]

print("Downloading price data from Alpha Vantage...")
all_tickers = universe_tickers + [benchmark_ticker]
price_data_dict = {}
for tk in all_tickers:
    print(f"Fetching: {tk}")
    series_close = fetch_alpha_vantage_data(tk, API_KEY)
    price_data_dict[tk] = series_close

df_prices = pd.DataFrame(price_data_dict).sort_index()
# Forward-fill minor gaps, then drop any rows still containing NaN
df_prices.ffill()
df_prices.dropna(how='any', inplace=True)

if benchmark_ticker not in df_prices.columns:
    raise ValueError(f"Benchmark {benchmark_ticker} not found in the fetched data.")

benchmark_prices = df_prices[benchmark_ticker]
stock_prices = df_prices.drop(columns=[benchmark_ticker])

stock_returns = stock_prices.pct_change().dropna(how='all')
benchmark_returns = benchmark_prices.pct_change()

if len(stock_returns) < lookback_period:
    raise ValueError("Not enough data to have a full 1-year (252 days) lookback.")

dates = stock_returns.index

Fetching: AAPL
Fetching: MSFT
Fetching: AMZN
Fetching: GOOGL
Fetching: META
Fetching: JNJ
Fetching: JPM
Fetching: V
Fetching: PG
Fetching: XOM
Fetching: INTC
Fetching: KO
Fetching: WMT
Fetching: NVDA
Fetching: SPY


**3. Helper Function: Sharpe Ratio, Optimizer for Weights, Optimizer for Portfolio(Brute Force)**

In [42]:
def compute_sharpe(weights, mu, cov, risk_free=risk_free_daily):
    """
    Sharpe ratio = (portfolio_return - risk_free) / portfolio_std
    portfolio_return (mu) and risk_free are DAILY rates.
    """
    w = np.asarray(weights)
    ret = w.dot(mu) - risk_free
    var = w.dot(cov).dot(w)
    if var <= 0:
        # Degenerate case => no or negative variance
        return np.inf if ret > 0 else 0
    return ret / np.sqrt(var)

def optimize_weights(
    tickers,
    mu_vector,
    cov_matrix,
    risk_free=risk_free_daily,
    max_w=0.30,
    tolerance=1e-8,
):
    """
    Maximize the Sharpe ratio for a given subset of tickers, subject to:
      - Long-only: w_i >= 0
      - sum of weights = 1
      - Each weight <= max_w
    Uses `scipy.optimize.minimize` with SLSQP. If no feasible solution is found,
    falls back to a uniform weighting across these tickers.
    """
    # Number of assets
    n = len(mu_vector)
    if n == 0:
        return np.array([])

    # Negative Sharpe objective
    def neg_sharpe(w):
        return -compute_sharpe(w, mu_vector, cov_matrix, risk_free)

    # Initial guess = equal weights
    w0 = np.ones(n) / n

    # Bounds: each weight between 0 and max_w
    bounds = [(0, max_w)] * n

    # Constraint: sum(w) = 1
    constraints = {"type": "eq", "fun": lambda w: np.sum(w) - 1}

    # Call SLSQP
    res = minimize(
        neg_sharpe,
        w0,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"ftol": 1e-9, "maxiter": 200}
    )

    if res.success:
        w_opt = np.array(res.x)
        # In principle, if the solver is successful and constraints are feasible,
        # sum(w_opt) should be extremely close to 1. Still, we check small numeric drift.
        sum_w = w_opt.sum()
        if abs(sum_w - 1.0) < tolerance:
            return w_opt
        else:
            # If for some reason it's off, fallback to uniform among these tickers
            print("[Warning] SLSQP successful but sum of weights != 1 within tolerance.")
            return np.ones(n) / n
    else:
        # If SLSQP wasn't successful, fallback to uniform
        print("[Warning] SLSQP failed to find a feasible solution. Using uniform weights.")
        return np.ones(n) / n


def brute_force_best_portfolio(universe_tickers, returns_window, portfolio_size=8, rf_rate=risk_free_daily):
    """
    Among all combinations of `portfolio_size` from `universe_tickers`,
    find the combination + weighting that yields the highest Sharpe ratio.
    
    Returns: 
      best_subset (list), best_weights (np.array), best_sharpe (float)
    """
    best_sharpe = -999
    best_subset = None
    best_weights = None
    
    # Precompute once
    mu_vec_full = returns_window.mean()
    cov_mat_full = returns_window.cov()
    
    for subset in combinations(universe_tickers, portfolio_size):
        subset = list(subset)
        subset_mu = mu_vec_full[subset].values
        subset_cov = cov_mat_full.loc[subset, subset].values
        
        # optimize weights for that subset
        w_opt = optimize_weights(subset, subset_mu, subset_cov, risk_free=rf_rate, max_w=max_weight)
        sr = compute_sharpe(w_opt, subset_mu, subset_cov, risk_free=rf_rate)
        
        if sr > best_sharpe:
            best_sharpe = sr
            best_subset = subset
            best_weights = w_opt
    
    return best_subset, best_weights, best_sharpe

**4. Setup Initial Portfolio & Data Structrues**

In [43]:
start_index = lookback_period
initial_date = dates[start_index]

print(f"\nBacktest starts at {initial_date.date()} (index: {start_index}).\n")

# The first window of data: from (start_index - lookback_period) up to start_index-1
init_window = stock_returns.iloc[start_index - lookback_period : start_index].dropna(axis=1, how='any')

# We only consider the tickers that actually have full data in that window
valid_tickers_init = init_window.columns.tolist()

# Among valid tickers, do brute-force to find best combination
if len(valid_tickers_init) < portfolio_size:
    raise ValueError(f"Not enough valid tickers ({len(valid_tickers_init)}) to pick {portfolio_size} at start!")

current_portfolio, w_init, best_sharpe_init = brute_force_best_portfolio(
    valid_tickers_init, init_window, portfolio_size=portfolio_size, rf_rate=risk_free_daily
)
current_weights = pd.Series(w_init, index=current_portfolio)

print("Initial Portfolio:", current_portfolio)
print("Initial Weights (capped @30%):", current_weights.round(3).to_dict())
print(f"Initial Sharpe: {best_sharpe_init:.3f}") # Daily Sharpe Ratio

# We'll track daily portfolio value, daily stock allocations, etc.
portfolio_value = 1.0
portfolio_values = []
portfolio_dates = []
weight_history = {}

all_stocks_universe = stock_prices.columns.tolist()


Backtest starts at 2013-05-23 (index: 252).

Initial Portfolio: ['GOOGL', 'META', 'JNJ', 'JPM', 'V', 'PG', 'KO', 'WMT']
Initial Weights (capped @30%): {'GOOGL': 0.127, 'META': 0.0, 'JNJ': 0.3, 'JPM': 0.14, 'V': 0.212, 'PG': 0.065, 'KO': 0.0, 'WMT': 0.155}
Initial Sharpe: 0.196


**5. Main Backtest Loop (Day-by-day)**

In [ ]:
for t in range(start_index, len(dates)):
    date = dates[t]
    portfolio_dates.append(date)
    portfolio_values.append(portfolio_value)

    # Store daily weights for plotting
    day_weight_series = pd.Series(0.0, index=all_stocks_universe)
    day_weight_series.loc[current_portfolio] = current_weights.values
    weight_history[date] = day_weight_series

    # Rebalance every `rebalance_interval` days
    if (t - start_index) % rebalance_interval == 0:
        window_start = max(0, t - lookback_period)
        # returns in the lookback window
        window_data = stock_returns.iloc[window_start : t].dropna(axis=1, how='any')
        valid_tickers = window_data.columns.tolist()
        
        if len(valid_tickers) >= portfolio_size:
            new_portfolio, new_weights, new_sharpe = brute_force_best_portfolio(
                valid_tickers, window_data, portfolio_size=portfolio_size, rf_rate=risk_free_daily
            )
            current_portfolio = new_portfolio
            current_weights = pd.Series(new_weights, index=new_portfolio)
            print(f"[{date.date()}] Rebalanced. New Sharpe: {new_sharpe:.3f}  |  Portfolio: {new_portfolio}")
        else:
            print(f"[{date.date()}] Not enough tickers with full data to form a {portfolio_size}-stock portfolio.")

    # compute daily return
    if t > 0 and len(current_portfolio) > 0:
        prev_date = dates[t - 1]
        prev_prices = stock_prices.loc[prev_date, current_portfolio]
        today_prices = stock_prices.loc[date, current_portfolio]
        daily_ret_assets = (today_prices - prev_prices) / prev_prices
        daily_ret_assets = daily_ret_assets.fillna(0)
        day_ret = np.dot(current_weights.values, daily_ret_assets.values)
    else:
        day_ret = 0.0

    portfolio_value *= (1 + day_ret)

# Create a final pandas Series with daily portfolio values
portfolio_series = pd.Series(portfolio_values, index=pd.DatetimeIndex(portfolio_dates)).sort_index()

**6. Performance Metrics & Comparison**

In [45]:
# align benchmark
benchmark_aligned = benchmark_prices.reindex(portfolio_series.index, method='ffill')
spx_norm = benchmark_aligned / benchmark_aligned.iloc[0] * portfolio_series.iloc[0]

port_daily_ret = portfolio_series.pct_change().dropna()
spx_daily_ret = spx_norm.pct_change().dropna()

total_return_port = portfolio_series.iloc[-1] / portfolio_series.iloc[0] - 1
total_return_spy = spx_norm.iloc[-1] / spx_norm.iloc[0] - 1

days = (portfolio_series.index[-1] - portfolio_series.index[0]).days
years = days / 365.25 if days > 0 else 0

cagr_port = (1 + total_return_port)**(1 / years) - 1 if years > 0 else np.nan
cagr_spy = (1 + total_return_spy)**(1 / years) - 1 if years > 0 else np.nan

vol_port = port_daily_ret.std() * sqrt(252)
vol_spy = spx_daily_ret.std() * sqrt(252)

sharpe_port = (port_daily_ret.mean() / port_daily_ret.std() * sqrt(252)
               if port_daily_ret.std() != 0 else 0)
sharpe_spy = (spx_daily_ret.mean() / spx_daily_ret.std() * sqrt(252)
              if spx_daily_ret.std() != 0 else 0)

# Max Drawdown
cum_port = portfolio_series.cummax()
drawdown_port = portfolio_series / cum_port - 1
max_dd_port = drawdown_port.min()

cum_spy = spx_norm.cummax()
drawdown_spy = spx_norm / cum_spy - 1
max_dd_spy = drawdown_spy.min()

print("\nPerformance Summary:")
print(f"Date range: {portfolio_series.index[0].date()} to {portfolio_series.index[-1].date()} ({years:.2f} yrs)")
print(f"Final Portfolio Value: {portfolio_series.iloc[-1]:.2f} "
      f"(Total Return: {total_return_port*100:.2f}%)")
print(f"Final SPY Value (normalized): {spx_norm.iloc[-1]:.2f} "
      f"(Total Return: {total_return_spy*100:.2f}%)")
print(f"CAGR - Portfolio: {cagr_port*100:.2f}%,  SPY: {cagr_spy*100:.2f}%")
print(f"Annual Volatility - Portfolio: {vol_port*100:.2f}%,  SPY: {vol_spy*100:.2f}%")
print(f"Sharpe Ratio - Portfolio: {sharpe_port:.2f},  SPY: {sharpe_spy:.2f}")
print(f"Max Drawdown - Portfolio: {max_dd_port*100:.2f}%,  SPY: {max_dd_spy*100:.2f}%")

# Plotly Visualizations

# 1) Portfolio vs. Benchmark
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=portfolio_series.index, y=portfolio_series,
                          mode='lines', name='Portfolio'))
fig1.add_trace(go.Scatter(x=spx_norm.index, y=spx_norm,
                          mode='lines', name='Benchmark (SPY)'))
fig1.update_layout(
    title="Portfolio Value vs. Benchmark (SPY)",
    xaxis_title="Date",
    yaxis_title="Value",
    hovermode="x unified"
)
fig1.show()

# 2) Daily Allocation Over Time (Stacked Area)
weight_df = pd.DataFrame(weight_history).T  # date x ticker
weight_df = weight_df.sort_index().fillna(0.0)

fig2 = go.Figure()
for stock in weight_df.columns:
    fig2.add_trace(go.Scatter(
        x=weight_df.index,
        y=weight_df[stock],
        stackgroup='one',
        mode='lines',
        name=stock
    ))
fig2.update_layout(
    title="Portfolio Allocation (Daily) - Each Stock",
    xaxis_title="Date",
    yaxis_title="Weight",
    hovermode="x unified"
)
fig2.show()

# 3) Drawdown Comparison
fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=drawdown_port.index,
    y=drawdown_port*100,
    mode='lines',
    name='Portfolio Drawdown'
))
fig3.add_trace(go.Scatter(
    x=drawdown_spy.index,
    y=drawdown_spy*100,
    mode='lines',
    name='SPY Drawdown'
))
fig3.update_layout(
    title="Drawdown (%) Over Time",
    xaxis_title="Date",
    yaxis_title="Drawdown (%)",
    hovermode="x unified"
)
fig3.show()


Performance Summary:
Date range: 2013-05-23 to 2025-02-28 (11.77 yrs)
Final Portfolio Value: 14.85 (Total Return: 1384.58%)
Final SPY Value (normalized): 4.43 (Total Return: 343.02%)
CAGR - Portfolio: 25.76%,  SPY: 13.48%
Annual Volatility - Portfolio: 20.77%,  SPY: 16.83%
Sharpe Ratio - Portfolio: 1.21,  SPY: 0.84
Max Drawdown - Portfolio: -25.84%,  SPY: -33.70%
